<a href="https://colab.research.google.com/github/ssvadla/Bridge_Project/blob/main/Supervised_Light_XG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTrain.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['Unnamed: 0', 'Skew', 'Number_Spans', 'Length_Max_Span',
       'Main_Design_Type', 'Deck_Design_Type', 'Wearing_Surface',
       'Deck_Protection', 'ADT', 'ADTT', 'State', 'Structure_Number',
       'Latitude', 'Longitude', 'Deck_Condition', 'Superstructure_Condition',
       'Substructure_Condition', 'Days_MinLT_0F', 'Days_MaxGT_90F',
       'Avg_Wind_Speed', 'Peak_Wind_Speed', 'Precipitation', 'Reconstructed',
       'Age', 'Deck_Area'],
      dtype='object')

In [4]:
Excluded_columns = ['State', 'Structure_Number', 'Latitude', 'Longitude', 'Superstructure_Condition', 'Substructure_Condition']

In [5]:
train = train.drop(columns = Excluded_columns)

In [7]:
train.shape

(41902, 19)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Main_Design_Type_Encoded'] = le.fit_transform(train['Main_Design_Type'])
train['Deck_Design_Type_Encoded'] = le.fit_transform(train['Deck_Design_Type'])
train['Wearing_Surface_Encoded'] = le.fit_transform(train['Wearing_Surface'])
train['Deck_Protection_Encoded'] = le.fit_transform(train['Deck_Protection'])

In [9]:
import lightgbm as lgb
classifier_lgb = lgb.LGBMClassifier()

In [10]:
X = train.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection']) 


In [11]:
y = train['Deck_Condition']

In [13]:
X.shape

(41902, 18)

In [12]:
classifier_lgb.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
test = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTest.csv')

In [19]:
test = test.drop(columns = Excluded_columns)

In [20]:
test['Main_Design_Type_Encoded'] = le.fit_transform(test['Main_Design_Type'])
test['Deck_Design_Type_Encoded'] = le.fit_transform(test['Deck_Design_Type'])
test['Wearing_Surface_Encoded'] = le.fit_transform(test['Wearing_Surface'])
test['Deck_Protection_Encoded'] = le.fit_transform(test['Deck_Protection'])

In [21]:
X_test = test.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection']) 


In [22]:
y_test = test['Deck_Condition']

In [23]:
X_test.shape

(13970, 18)

In [24]:
test_pred = classifier_lgb.predict(X_test)

In [25]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test,test_pred, digits=4)

In [26]:
print(classification_report)

              precision    recall  f1-score   support

           6     0.5091    0.5097    0.5094      3861
           7     0.5818    0.6115    0.5963      4453
          45     0.5990    0.5830    0.5909      2746
          89     0.7528    0.7117    0.7317      2910

    accuracy                         0.5986     13970
   macro avg     0.6107    0.6040    0.6071     13970
weighted avg     0.6007    0.5986    0.5994     13970



In [28]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
classifier_xg = XGBClassifier()
classifier_xg.fit(X, y)
test_pred_xg = classifier_xg.predict(X_test)
classification_report_xg = classification_report(y_test,test_pred_xg, digits=4)
print(classification_report_xg)

              precision    recall  f1-score   support

           6     0.4727    0.5183    0.4944      3861
           7     0.5652    0.5774    0.5712      4453
          45     0.5731    0.5124    0.5410      2746
          89     0.7263    0.6821    0.7035      2910

    accuracy                         0.5701     13970
   macro avg     0.5843    0.5725    0.5776     13970
weighted avg     0.5747    0.5701    0.5716     13970

